In [0]:
# spark.sql("use catalog hp_az_catalog")
# spark.sql("use schema hp_iris_ml")

In [0]:
dbutils.widgets.text("source_path", "abfss://deltalake@oneenvadls.dfs.core.windows.net/hema_dir/models")
dbutils.widgets.text("volume_path", "/Volumes/hp_az_catalog/hp_iris_ml/hp_ml_vol")
dbutils.widgets.text("checkpoint_path", "/Volumes/hp_az_catalog/hp_iris_ml/hp_ml_vol/checkpoints")

In [0]:
source_path = dbutils.widgets.get("source_path")
volume_path = dbutils.widgets.get("volume_path")
checkpoint_path = dbutils.widgets.get("checkpoint_path")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, current_timestamp
import os

spark = SparkSession.builder.getOrCreate()
df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "binaryFile")
    .option("cloudFiles.includeExistingFiles", "true")  
    .option("pathGlobFilter", "*.pkl")  # only .pkl files
    .load(source_path)
    .withColumn("ingest_time", current_timestamp())
)

def write_binary_to_volume(batch_df, batch_id):
    import os
    for row in batch_df.collect():
        file_name = os.path.basename(row["path"])
        target_path = os.path.join(volume_path, file_name)
        content = row["content"]
        with open(target_path, "wb") as f:
            f.write(content)

query = (
    df.writeStream
    .foreachBatch(write_binary_to_volume)
    .option("checkpointLocation", checkpoint_path)
    .trigger(once=True)
    .start()
)
query.awaitTermination()